https://dacon.io/competitions/official/235864/codeshare/4229?page=1&dtype=recent


In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_excel('news_label.xlsx', engine='openpyxl')

In [3]:
df = df.rename(columns={'dates':'date', 'titles':'title','contents':'content'})
df['content'] = df['title'] + df['content']

In [4]:
df = df.drop(columns=['근거', 'title'])

In [5]:
df['label'] = df['label'].astype(int)
df

,date,content,label
0,2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//[\n마스카라까지 지우...,1
1,2021.10.07,"[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//[""파괴적 커머스 시대...",1
2,2021.10.08,[株머니 매니저의 HOT종목]//[\n\n\n\n\n 씨에스윈드는 풍력발전기를 구성...,1
3,2021.10.11,"[김민수 더맘마 대표 ""동네마트와 '상생'이 경쟁력…30분 총알배송 목표""]//[\...",1
4,2021.10.13,"[""카카오 보고있나""…진격의 네이버페이, 中 최대 직구몰 손잡았다]//[[주간투자동...",1
...,...,...,...
274,2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//[\n심사 승인 났는...",0
275,2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",1
276,2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//[\n■국감 오르...",0
277,2022.09.29,[국내 이커머스 상장 1호 주인공 누가될까 [격변의 이커머스①]]//[\n이커머스 ...,0


### 데이터 정제하기

In [6]:
# 문자 외의 것들 제거
df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
# 혹시 모를 다중 공백 제거
df['content'] = df['content'].str.replace(' +',' ') 
df

/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_3401/2956849729.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_3401/2956849729.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(' +',' ')


,date,content,label
0,2021.10.05,톡톡 이상품 한빛코리아 다지워 네이처 클렌징 솝 마스카라까지 지우는 세정비누 피부...,1
1,2021.10.07,격동의 유통가 편의점 성장 정체 미래는 근거리 쇼핑 파괴적 커머스 시대 데이터 경...,1
2,2021.10.08,머니 매니저의 HOT종목 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 ...,1
3,2021.10.11,김민수 더맘마 대표 동네마트와 상생 이 경쟁력 30분 총알배송 목표 별도 물류센터...,1
4,2021.10.13,카카오 보고있나 진격의 네이버페이 최대 직구몰 손잡았다 주간투자동향 파운트 400...,1
...,...,...,...
274,2022.09.23,WCP 흥행 실패에 컬리 케이뱅크 상장 시기 안갯속 심사 승인 났는데 IPO 감감...,0
275,2022.09.26,먹거리 IT 네이처랩 오경주 대표 맛있고 건강한 주스 추천 서비스를 개발합니다 서...,1
276,2022.09.27,투자 늘렸는데 이제와서 퀵커머스마저 규제하나 초긴장 국감 오르는 유통업 새 먹거리...,0
277,2022.09.29,국내 이커머스 상장 1호 주인공 누가될까 격변의 이커머스 이커머스 IPO 대전 흑...,0


In [7]:
train = df

In [8]:
train, test = train_test_split(train)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
test.reset_index(inplace=True)

In [9]:
test

,index,date,content,label
0,256,2022.08.31,새벽배송 전쟁 전열 재정비 시장 판도 변할까 이랜드 새벽배송 오아시스와 손잡아 신...,1
1,146,2022.04.01,현금흐름 악화 컬리 몸값 4조 6조 쿠팡 아냐 오버밸류 주의보 중소기업유통센터 민...,0
2,105,2022.02.06,넘버스 패션 플랫폼 연 거래액 1조원 시대 의 의미 시간이 없다면무신사가 연 거래...,1
3,143,2022.03.29,800리터 양문형 냉장고도 내일 배송 G마켓의 새벽 은 다를까 누가 더 멀리 가나...,1
4,107,2022.02.08,쓱닷컴 마켓컬리 오아시스 올 IPO시장 풍향계 오아시스마켓 홈앤쇼핑서 100억원 ...,1
...,...,...,...,...
65,182,2022.05.17,드디어 요기요도 참전 불붙은 퀵커머스 전쟁 요기요 GS리테일 요마트 부활 퀵커머스...,0
66,27,2021.11.11,기념일마다 선물에 월급날엔 일찍 집으로 MZ가 픽 한 플랫폼 5총사 복지도 톡톡 ...,1
67,73,2022.01.02,2022년 IPO 대어 몰려온다 LG에너지솔루션 현대엔지니어링 쓱닷컴 상장 예고 ...,1
68,209,2022.06.28,고물가 시대 합리적 소비 주목 온 오프 유통업계 진땀 DD 퇴근길 아이폰14 양산...,1


from sklearn.model_selection import train_test_split
X = df['content']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

X_train

X_test

### 토큰화 및 TF-IDF(Term Frequency-Inverse Document Frequency) 벡터화

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

from sklearn.linear_model import LinearRegression

In [22]:
test= test.drop(['label','index'],axis=1)

In [23]:
test

,date,content
0,2022.08.31,새벽배송 전쟁 전열 재정비 시장 판도 변할까 이랜드 새벽배송 오아시스와 손잡아 신...
1,2022.04.01,현금흐름 악화 컬리 몸값 4조 6조 쿠팡 아냐 오버밸류 주의보 중소기업유통센터 민...
2,2022.02.06,넘버스 패션 플랫폼 연 거래액 1조원 시대 의 의미 시간이 없다면무신사가 연 거래...
3,2022.03.29,800리터 양문형 냉장고도 내일 배송 G마켓의 새벽 은 다를까 누가 더 멀리 가나...
4,2022.02.08,쓱닷컴 마켓컬리 오아시스 올 IPO시장 풍향계 오아시스마켓 홈앤쇼핑서 100억원 ...
...,...,...
65,2022.05.17,드디어 요기요도 참전 불붙은 퀵커머스 전쟁 요기요 GS리테일 요마트 부활 퀵커머스...
66,2021.11.11,기념일마다 선물에 월급날엔 일찍 집으로 MZ가 픽 한 플랫폼 5총사 복지도 톡톡 ...
67,2022.01.02,2022년 IPO 대어 몰려온다 LG에너지솔루션 현대엔지니어링 쓱닷컴 상장 예고 ...
68,2022.06.28,고물가 시대 합리적 소비 주목 온 오프 유통업계 진땀 DD 퇴근길 아이폰14 양산...


ValueError: could not convert string to float: ' 마켓인 밸류업 올인 이머커스 3대장 IPO 앞두고 몸집 키우기 새벽배송 이커머스 3대장 IPO 동시출격서로 강점 어필하며 몸값 불리기 총력전SSG 컬리는 성장 오아시스 수익성 부각 시장 평가서 객관적인 장단점 드러날 것 이데일리 김성훈 기자 국내 새벽 배송 이커머스 전자상거래 3대 주자로 꼽히는 SSG 쓱 닷컴과 마켓컬리 오아시스의 기업공개 IPO 가 내년으로 결정된 가운데 회사마다 몸값 불리기 작업이 한창이다 쓱닷컴은 수조원대 인수합병 M A 마켓컬리는 전년 대비 2배 가까운 매출 신장에 따른 외형 성장을 어필하고 있다 오아시스는 오프라인 매장 증설로 수익성 사수라는 키워드를 새기는 데 주력하는 모습이다 상장 출사표와 함께 시장에서 점친 밸류에이션 기업가치 에 변화의 조짐이 일어날지가 관건으로 꼽힌다 그래픽 김일환 기자 IPO 선언 후 몸값 불리기 나선 새벽배송 한국투자증권에 따르면 내년도 신규 상장이 거론되는 예상 기업가치 시가총액 1조원 이상 기업 수는 13개로 나타났다 올해 상장한 1조원 이상 기업 11곳을 웃도는 수치다 공교롭게도 내년 상장을 선언한 새벽 배송 이커머스 업체 3곳 모두 1조원 이상의 기업가치로 시장 분위기를 좌우할 공모 유망주로 꼽히고 있다 쓱닷컴의 기업가치가 10조원으로 점쳐지는 가운데 마켓컬리 운영사인 컬리가 5조원 오아이스가 1조원으로 기대주에 이름을 올렸다 흥미로운 점은 이들 세 업체 모두 시장에서 점치는 예상 기업가치에 만족하지 않는 모습이라는 것이다 남은 시간 동안 자신들의 강점을 어필해 몸값을 끌어올리겠다는 계산이다 공모가 산정에다 청약 흥행 상장 후 기업가치로 이어지는 과정이 회사가 거머쥘 공모자금 규모와도 이어지기 때문이다 몸값 10조원이 점쳐지는 쓱닷컴은 투자 유치를 조건으로 재무적투자자 FI 들과 맺은 IPO 기한을 1년가량 앞당기는 여유를 보이고 있다 신세계그룹의 온라인 통합 서비스 라는 자신감이 더해지며 상장 주관사 선정에서 

In [ ]:
lin

In [34]:
for max_features in range(100, 3000, 100):
    vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

    X = train['content']
    y = train['label']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
    
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    
    predicted = lin_reg.predict(X_test)
    
    print(f"Linear Regression      max_features: {max_features}, Accuracy: {lin_reg.predict(X_train)}" )
    

Linear Regression      max_features: 100, Accuracy: [ 0.14604874  0.44352439  0.62793164  0.13436603  0.40481274 -0.02787301
 -0.25680194  0.96656957  0.90053314  0.95562017  0.15698651 -0.04817871
  0.62542222  0.19293273  0.92400903  0.93011064  1.06077714  0.79321814
  1.00216571  0.95266067  0.55783852  0.61430384  0.83377613  0.80300151
  1.02874206  0.32613601  0.8291873   0.9877387   0.94500092  0.87587841
  0.54810593  0.63445642  0.41585664  0.05268727  0.96672274  0.40316034
  0.27567249  1.10185173  0.6076116   1.21743578  0.86288176  0.48504154
  0.81685872  0.03239677  0.94584031  1.13545627  0.69985468  0.66679826
  0.8023356   0.16194797  0.94693801 -0.35136907  0.60073167  1.33552607
  0.08181991  0.63135798  0.76619331  0.72244864  0.62896626  1.06861933
  0.70700143  0.54093619  1.10168902  0.84393047  0.39140056  0.87932803
  1.11659894  0.43977302  0.60157639  1.39650866  0.07364957  0.91402466
  0.56668088  0.99854944  0.88934009  0.8056613   1.16734849 -0.06412546

Linear Regression      max_features: 500, Accuracy: [-1.67157013e-05 -3.08601811e-05  1.00001602e+00  7.80015007e-06
  5.38891073e-05  3.17954657e-05 -3.05367353e-05  9.99982935e-01
  1.00003610e+00  9.99968190e-01 -4.83427943e-06  1.77019620e-05
  5.19198509e-05 -5.04109789e-05  1.00000464e+00  9.99995710e-01
  9.99972456e-01  9.99986289e-01  1.00001770e+00  1.00000411e+00
 -7.21367351e-05  1.00002055e+00  1.00003591e+00  9.99983793e-01
  1.00004441e+00 -4.84511650e-05  1.00004178e+00  1.00001971e+00
  9.99996912e-01  1.00001555e+00  1.00002446e+00 -5.14062827e-05
  1.18707462e-04 -2.51917263e-05  1.00001737e+00  3.62283907e-05
  2.53142359e-05  9.99969281e-01 -5.21781038e-05  1.00001343e+00
  9.99988256e-01  2.88814053e-05  9.99972408e-01 -9.36494997e-06
  9.99991629e-01  9.99983686e-01  1.00000280e+00  1.00002082e+00
  1.00000088e+00 -2.70786407e-05  9.99984854e-01 -1.89457651e-05
  1.00000794e+00  1.00003584e+00 -1.08066585e-05 -5.65228840e-05
  9.99964745e-01  1.00000975e+00  1.00

Linear Regression      max_features: 800, Accuracy: [-6.05577482e-06  2.52109315e-05  9.99989177e-01 -7.72804185e-06
  1.24344106e-05 -4.59728364e-07 -9.13277531e-06  1.00000380e+00
  1.00000566e+00  9.99988601e-01 -2.11267555e-05 -1.66209533e-05
 -9.31740131e-06 -2.82872388e-05  9.99995689e-01  1.00000097e+00
  1.00004191e+00  9.99991582e-01  9.99998933e-01  9.99990930e-01
  1.47225024e-05  1.00001507e+00  1.00003078e+00  1.00000437e+00
  1.00000461e+00  3.78027320e-06  1.00000476e+00  1.00000900e+00
  9.99997008e-01  1.00000041e+00  1.00000288e+00  2.21448788e-05
  4.21980493e-05 -3.45722183e-06  1.00000470e+00  1.57525168e-05
 -1.48506272e-06  9.99999691e-01 -4.34464321e-05  9.99997935e-01
  1.00000425e+00  1.83151555e-05  9.99989720e-01 -1.17729806e-05
  1.00000460e+00  1.00001267e+00  1.00004483e+00  1.00000293e+00
  1.00001668e+00  1.43663387e-06  1.00002108e+00  9.38958241e-06
  1.00000453e+00  9.99993974e-01  2.12367149e-05  3.84439554e-05
  1.00000314e+00  9.99992722e-01  9.99

Linear Regression      max_features: 1100, Accuracy: [ 1.59611189e-06  9.06793682e-06  1.00000984e+00 -4.55938796e-06
 -8.67614563e-06 -3.26595937e-06 -3.05413480e-06  1.00000610e+00
  9.99977758e-01  9.99999304e-01  2.84944893e-06  1.26999850e-06
 -4.96834794e-06 -8.50908578e-06  9.99993618e-01  1.00000912e+00
  1.00001221e+00  1.00000250e+00  9.99997741e-01  9.99992528e-01
 -3.93938335e-06  9.99996083e-01  1.00000642e+00  1.00001944e+00
  1.00000306e+00  5.51423012e-06  1.00001431e+00  1.00000074e+00
  9.99994575e-01  9.99999294e-01  9.99989039e-01  2.06958229e-05
 -1.85291034e-05  4.72183575e-06  9.99998762e-01  1.84100197e-06
 -1.45506441e-06  1.00000220e+00  5.46366812e-05  9.99998948e-01
  9.99999526e-01 -1.33908272e-05  1.00000850e+00 -2.48262952e-06
  9.99994269e-01  1.00000539e+00  1.00002359e+00  9.99985070e-01
  9.99993492e-01 -2.74619823e-06  9.99997862e-01 -5.77658109e-06
  9.99984698e-01  9.99993659e-01  7.28825389e-06  3.84116959e-06
  9.99992476e-01  1.00001167e+00  1.0

Linear Regression      max_features: 1400, Accuracy: [-1.59143615e-06 -7.62813626e-06  9.99995538e-01  7.22941158e-07
 -7.08909989e-06 -3.68071418e-07  7.86960220e-08  9.99999893e-01
  1.00000548e+00  9.99994875e-01  1.44377092e-05 -5.88497793e-06
  3.56471001e-07 -5.88874983e-06  1.00000343e+00  1.00001145e+00
  9.99993383e-01  1.00001780e+00  9.99993096e-01  1.00000501e+00
 -9.63111570e-06  9.99998630e-01  1.00001048e+00  1.00000309e+00
  1.00000431e+00 -1.16320245e-06  1.00000017e+00  9.99991105e-01
  1.00000784e+00  9.99992826e-01  1.00001820e+00 -7.14529268e-06
  1.34457202e-06 -7.34341299e-07  9.99992814e-01  9.03530416e-06
  4.57457294e-07  1.00000668e+00 -1.44496120e-05  9.99998350e-01
  9.99992708e-01  6.26766730e-07  9.99993979e-01  3.16417667e-06
  9.99994046e-01  9.99991627e-01  1.00003090e+00  1.00000251e+00
  1.00000157e+00 -9.59054961e-06  1.00000033e+00  5.26614121e-06
  1.00000919e+00  9.99990565e-01  9.71661818e-06  1.34248620e-05
  9.99993346e-01  1.00001831e+00  1.0

Linear Regression      max_features: 1700, Accuracy: [ 1.70702733e-06 -1.71340595e-05  1.00000353e+00 -7.26522421e-07
  1.35512725e-06  2.71777200e-06  9.32825080e-06  9.99998179e-01
  1.00000388e+00  1.00000170e+00  5.04814425e-06  1.60705677e-06
  2.02293091e-06 -8.25666428e-07  1.00000331e+00  1.00000629e+00
  1.00000536e+00  9.99995140e-01  9.99994464e-01  9.99998943e-01
 -6.90540229e-06  9.99983284e-01  9.99995370e-01  9.99999992e-01
  9.99998122e-01 -5.88961901e-06  1.00000360e+00  1.00000853e+00
  9.99996999e-01  1.00000128e+00  1.00000479e+00 -1.12854559e-05
  1.80083302e-05 -1.09240123e-06  1.00001489e+00  3.06658933e-06
 -5.05374263e-06  1.00000614e+00 -1.95148510e-05  9.99999356e-01
  9.99994437e-01  5.60662003e-06  9.99999818e-01  3.92360078e-06
  9.99998297e-01  1.00000591e+00  1.00001891e+00  1.00000428e+00
  1.00000087e+00 -6.10409942e-06  1.00000240e+00  4.06316140e-06
  1.00000972e+00  9.99999963e-01 -1.16102954e-07  2.51808069e-06
  9.99990981e-01  1.00000238e+00  1.0

Linear Regression      max_features: 2000, Accuracy: [-9.07722768e-06 -3.17465111e-06  9.99996081e-01  9.83183823e-07
  1.00493995e-05  1.86691220e-06  8.50453699e-06  1.00000513e+00
  1.00000519e+00  9.99995952e-01 -2.77725737e-06  1.39605119e-06
 -4.88374618e-06 -1.26210444e-06  9.99995742e-01  9.99998352e-01
  1.00000288e+00  1.00000263e+00  9.99998223e-01  1.00000593e+00
 -5.31322756e-06  9.99983667e-01  1.00000729e+00  9.99996178e-01
  1.00000295e+00 -3.48662392e-06  9.99999135e-01  9.99992012e-01
  9.99996995e-01  9.99988670e-01  1.00000647e+00 -5.78189135e-06
  1.44759600e-05  9.29169660e-07  9.99999209e-01  1.38203238e-06
  3.27828819e-06  9.99996452e-01 -1.11081862e-05  9.99997182e-01
  1.00000587e+00  9.96268548e-07  1.00000857e+00  4.14229295e-06
  1.00000469e+00  1.00000252e+00  1.00003736e+00  9.99999688e-01
  9.99997767e-01  2.14574931e-06  1.00000198e+00 -3.55074159e-06
  9.99992668e-01  9.99991846e-01 -7.94171537e-06 -2.04644155e-05
  1.00000545e+00  1.00000619e+00  1.0

Linear Regression      max_features: 2300, Accuracy: [-1.93818285e-06  4.47496400e-06  1.00000357e+00 -2.64827953e-06
  4.64580937e-06 -3.78900660e-06  9.23657659e-06  9.99995094e-01
  9.99995133e-01  1.00000272e+00  1.85756886e-06 -4.56021824e-07
  3.68630153e-06 -6.11616479e-06  9.99997124e-01  9.99997314e-01
  9.99995777e-01  9.99999256e-01  1.00000599e+00  1.00000451e+00
 -4.94118981e-06  9.99990220e-01  1.00000687e+00  1.00000018e+00
  9.99997878e-01  5.32138078e-06  9.99996407e-01  9.99997616e-01
  1.00000644e+00  1.00000272e+00  1.00000093e+00 -5.76634301e-07
 -5.99126384e-06  3.64010964e-06  9.99994464e-01 -4.32074808e-06
  1.48272043e-06  9.99994900e-01 -6.34032437e-06  1.00000058e+00
  9.99996655e-01 -1.53864632e-06  1.00000873e+00 -6.10467892e-06
  1.00000363e+00  1.00000303e+00  1.00001969e+00  1.00000024e+00
  9.99996677e-01  1.59046742e-06  1.00000024e+00  9.07216641e-07
  1.00000262e+00  9.99998042e-01  4.28000737e-07  5.38818733e-06
  1.00000329e+00  9.99996514e-01  1.0

Linear Regression      max_features: 2600, Accuracy: [ 1.29877295e-06  9.69090546e-06  9.99999639e-01  7.58893644e-07
  6.67623349e-06 -6.48934230e-06  2.08148218e-06  1.00000228e+00
  9.99994936e-01  9.99998994e-01 -3.66411785e-06  2.16475837e-06
  1.71057982e-06 -4.50816045e-06  1.00000544e+00  1.00000583e+00
  1.00000211e+00  9.99998453e-01  1.00000342e+00  1.00000233e+00
 -2.08566707e-06  9.99998223e-01  1.00000440e+00  9.99999829e-01
  9.99990394e-01  4.14859598e-06  9.99995654e-01  9.99999663e-01
  1.00000230e+00  9.99998397e-01  1.00000052e+00 -1.98005753e-07
 -6.58106391e-06  4.16326810e-06  9.99997236e-01  5.01810277e-07
  1.09667176e-06  9.99998600e-01 -1.82219794e-05  1.00000201e+00
  9.99997243e-01  3.98089272e-06  9.99999294e-01  1.70875275e-06
  1.00000059e+00  9.99995700e-01  1.00002295e+00  1.00000074e+00
  9.99992630e-01 -3.25835343e-07  9.99999375e-01 -3.20561422e-06
  9.99997140e-01  9.99995563e-01  5.22862390e-06  8.60142151e-06
  1.00000502e+00  9.99992517e-01  9.9

Linear Regression      max_features: 2900, Accuracy: [-3.92280340e-06  7.06061689e-06  9.99998837e-01 -5.27383096e-07
  5.40351948e-06 -2.21629752e-06  1.82222832e-06  1.00000017e+00
  9.99998526e-01  1.00000093e+00 -4.97103163e-06  1.63235308e-06
  3.91809260e-07 -4.17074491e-07  1.00000540e+00  1.00000925e+00
  1.00000086e+00  1.00000366e+00  9.99999830e-01  9.99999908e-01
  5.13374474e-06  1.00000346e+00  1.00000670e+00  1.00000161e+00
  1.00000205e+00  6.51977616e-07  9.99999805e-01  1.00000178e+00
  9.99999457e-01  9.99993356e-01  1.00000585e+00 -2.20879589e-06
 -9.40150762e-06 -1.40267837e-06  9.99997883e-01  2.07523749e-06
 -1.78026981e-06  9.99998083e-01 -1.36704354e-05  9.99997259e-01
  1.00000005e+00  7.12098416e-07  1.00000037e+00  1.64287657e-06
  1.00000025e+00  9.99996057e-01  1.00000703e+00  9.99996230e-01
  9.99998314e-01  2.00501428e-06  1.00000216e+00  1.37865603e-06
  9.99995749e-01  1.00000233e+00 -1.67929242e-06 -3.90284460e-06
  1.00000387e+00  9.99995803e-01  9.9

In [37]:
from sklearn.metrics import mean_squared_error

y_train = lin_reg.predict(y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[1 0 1 1 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1
 0 1 1 1 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [39]:
# tf-idf 벡터화
for max_features in range(100, 10000, 100):
    vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

    X = train['content']
    y = np.array(train.label)

    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

    X_train = vectorizer.fit_transform(X_train)
    X_eval = vectorizer.transform(X_eval)
    
    lgs = LogisticRegression(class_weight='balanced')
    naive = BernoulliNB()
    sgd = SGDClassifier(random_state=42, n_jobs=-1)
    rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
    svc = SVC(random_state=42)
    ada = AdaBoostClassifier(random_state=42)
    
    lgs.fit(X_train, y_train)
    naive.fit(X_train, y_train)
    sgd.fit(X_train, y_train)
    rfc.fit(X_train, y_train)
    svc.fit(X_train, y_train)
    ada.fit(X_train, y_train)
    
    predicted = lgs.predict(X_eval)
    predicted = naive.predict(X_eval)
    predicted = sgd.predict(X_eval)
    predicted = rfc.predict(X_eval)
    predicted = svc.predict(X_eval)
    predicted = ada.predict(X_eval)
    
    print(f"1. Logistic_egression      max_features: {max_features}, Accuracy: {lgs.score(X_eval, y_eval)}" )
    print(f"2. Naive_bayse             max_features: {max_features}, Accuracy: {naive.score(X_eval, y_eval)}" )
    print(f"3. SGD                     max_features: {max_features}, Accuracy: {sgd.score(X_eval, y_eval)}" )
    print(f"4. RandomForestClassifier  max_features: {max_features}, Accuracy: {rfc.score(X_eval, y_eval)}" )
    print(f"5. SVC                     max_features: {max_features}, Accuracy: {svc.score(X_eval, y_eval)}" )
    print(f"6. AdaBoost                max_features: {max_features}, Accuracy: {ada.score(X_eval, y_eval)}" )
    print('=======')

1. Logistic_egression      max_features: 100, Accuracy: 0.8095238095238095
2. Naive_bayse             max_features: 100, Accuracy: 0.5476190476190477
3. SGD                     max_features: 100, Accuracy: 0.5
4. RandomForestClassifier  max_features: 100, Accuracy: 0.6190476190476191
5. SVC                     max_features: 100, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 100, Accuracy: 0.6428571428571429
1. Logistic_egression      max_features: 200, Accuracy: 0.6190476190476191
2. Naive_bayse             max_features: 200, Accuracy: 0.5952380952380952
3. SGD                     max_features: 200, Accuracy: 0.5714285714285714
4. RandomForestClassifier  max_features: 200, Accuracy: 0.5952380952380952
5. SVC                     max_features: 200, Accuracy: 0.5238095238095238
6. AdaBoost                max_features: 200, Accuracy: 0.5714285714285714
1. Logistic_egression      max_features: 300, Accuracy: 0.6428571428571429
2. Naive_bayse             max_features:

1. Logistic_egression      max_features: 1900, Accuracy: 0.7380952380952381
2. Naive_bayse             max_features: 1900, Accuracy: 0.5952380952380952
3. SGD                     max_features: 1900, Accuracy: 0.6904761904761905
4. RandomForestClassifier  max_features: 1900, Accuracy: 0.6190476190476191
5. SVC                     max_features: 1900, Accuracy: 0.5952380952380952
6. AdaBoost                max_features: 1900, Accuracy: 0.6666666666666666
1. Logistic_egression      max_features: 2000, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 2000, Accuracy: 0.5952380952380952
3. SGD                     max_features: 2000, Accuracy: 0.5476190476190477
4. RandomForestClassifier  max_features: 2000, Accuracy: 0.7142857142857143
5. SVC                     max_features: 2000, Accuracy: 0.5952380952380952
6. AdaBoost                max_features: 2000, Accuracy: 0.5952380952380952
1. Logistic_egression      max_features: 2100, Accuracy: 0.7857142857142857
2. Naive_bay

1. Logistic_egression      max_features: 3700, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 3700, Accuracy: 0.6904761904761905
3. SGD                     max_features: 3700, Accuracy: 0.6428571428571429
4. RandomForestClassifier  max_features: 3700, Accuracy: 0.5952380952380952
5. SVC                     max_features: 3700, Accuracy: 0.5714285714285714
6. AdaBoost                max_features: 3700, Accuracy: 0.7380952380952381
1. Logistic_egression      max_features: 3800, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 3800, Accuracy: 0.6904761904761905
3. SGD                     max_features: 3800, Accuracy: 0.7142857142857143
4. RandomForestClassifier  max_features: 3800, Accuracy: 0.6190476190476191
5. SVC                     max_features: 3800, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 3800, Accuracy: 0.7142857142857143
1. Logistic_egression      max_features: 3900, Accuracy: 0.7619047619047619
2. Naive_bay

1. Logistic_egression      max_features: 5500, Accuracy: 0.7857142857142857
2. Naive_bayse             max_features: 5500, Accuracy: 0.6666666666666666
3. SGD                     max_features: 5500, Accuracy: 0.6428571428571429
4. RandomForestClassifier  max_features: 5500, Accuracy: 0.6428571428571429
5. SVC                     max_features: 5500, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 5500, Accuracy: 0.6190476190476191
1. Logistic_egression      max_features: 5600, Accuracy: 0.7857142857142857
2. Naive_bayse             max_features: 5600, Accuracy: 0.6666666666666666
3. SGD                     max_features: 5600, Accuracy: 0.6666666666666666
4. RandomForestClassifier  max_features: 5600, Accuracy: 0.6428571428571429
5. SVC                     max_features: 5600, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 5600, Accuracy: 0.6190476190476191
1. Logistic_egression      max_features: 5700, Accuracy: 0.7857142857142857
2. Naive_bay

1. Logistic_egression      max_features: 7300, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 7300, Accuracy: 0.6428571428571429
3. SGD                     max_features: 7300, Accuracy: 0.6904761904761905
4. RandomForestClassifier  max_features: 7300, Accuracy: 0.6190476190476191
5. SVC                     max_features: 7300, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 7300, Accuracy: 0.6190476190476191
1. Logistic_egression      max_features: 7400, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 7400, Accuracy: 0.6428571428571429
3. SGD                     max_features: 7400, Accuracy: 0.7142857142857143
4. RandomForestClassifier  max_features: 7400, Accuracy: 0.6428571428571429
5. SVC                     max_features: 7400, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 7400, Accuracy: 0.6666666666666666
1. Logistic_egression      max_features: 7500, Accuracy: 0.7619047619047619
2. Naive_bay

1. Logistic_egression      max_features: 9100, Accuracy: 0.7857142857142857
2. Naive_bayse             max_features: 9100, Accuracy: 0.6904761904761905
3. SGD                     max_features: 9100, Accuracy: 0.6904761904761905
4. RandomForestClassifier  max_features: 9100, Accuracy: 0.6904761904761905
5. SVC                     max_features: 9100, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 9100, Accuracy: 0.5714285714285714
1. Logistic_egression      max_features: 9200, Accuracy: 0.7857142857142857
2. Naive_bayse             max_features: 9200, Accuracy: 0.6904761904761905
3. SGD                     max_features: 9200, Accuracy: 0.6666666666666666
4. RandomForestClassifier  max_features: 9200, Accuracy: 0.6904761904761905
5. SVC                     max_features: 9200, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 9200, Accuracy: 0.5714285714285714
1. Logistic_egression      max_features: 9300, Accuracy: 0.7857142857142857
2. Naive_bay

In [47]:
from xgboost import XGBClassifier

X = train['content']
y = np.array(train.label)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

X_train = vectorizer.fit_transform(X_train)
X_eval = vectorizer.transform(X_eval)

# XGBClassifier 클래스의 객체 생성
xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=5)

# 모델 학습
xgb.fit(X_train, y_train, early_stopping_rounds=400, eval_set=evals, eval_metric="logloss",  verbose=True)

# 예측
predicted = xgb_wrapper.predict(X_eval)

# 평가 - 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_eval, pred)
print('인공 신경망 예측 정확도: {0:.4f}'.format(accuracy))


C:\Users\hslio\miniforge3\envs\lhs\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\hslio\miniforge3\envs\lhs\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


AssertionError: Must have at least 1 validation dataset for early stopping.

In [77]:
# 2600 - 3000 이 0.83으로 정확도가 가장 높게 나옴

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=2600)

X = train['content']
y = np.array(train.label)

X_train = vectorizer.fit_transform(X)
X_test = vectorizer.transform(test['content'])
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y)
predicted = lgs.predict(X_test)

X_train = vectorizer.fit_transform(train['content'])
X_test = vectorizer.transform(test['content'])
y = np.array(train.label)

# lgs = LogisticRegression(class_weight='balanced')
# lgs.fit(X_train, y)
# predicted = lgs.predict(X_test)
rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc.fit(X_train, y)
predicted = rfc.predict(X_test)

In [78]:
predicted

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1])

In [79]:
a = predicted.tolist()
b = np.array(test.label).tolist()

In [80]:
c = [i for i,j in zip(a,b) if i == j]
print(len(c))

51


In [81]:
correct_ratio = len(c) / len(test) * 100
correct_ratio

72.85714285714285

In [28]:
news_feature = df.drop(columns = ['content'])

In [30]:
news_feature.to_csv('news_feature.csv')